# Práctica 4

Dado el dataset Genealogía el cual está formado por: 

`<nombre_individuo, dni_individuo, dni_mamá>`

### c) El nombre de la “abuela” que tiene más descendientes

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import col
import time

sc = SparkContext("local", "My program")
sqlContext = SQLContext(sc)

In [2]:
genealogia = sc.textFile("datasets/genealogia.txt")
genealogia = genealogia.map(lambda t : t.split("\t"))

genealogia = genealogia.map(lambda t: 
                               Row(
                                   nombre_individuo = str(t[0]),
                                   dni_individuo = int(t[1]),
                                   dni_mama = int(t[2]) if t[2] != 'None' else None 
                               )
                           )

genealogiaDF = sqlContext.createDataFrame(genealogia)
genealogiaDF.registerTempTable("Genealogia")

In [3]:
def getAbuelas():
    query = f"""
    SELECT g.dni_individuo as dni_0, g.nombre_individuo as nombre
    FROM Genealogia g
    WHERE g.dni_mama is null"""
    result = sqlContext.sql(query)
    return result

def getHijos(tmp, i):
    tmp.registerTempTable("tmp")
    query = f"""
    SELECT tmp.*, g.dni_individuo as dni_{i + 1}
    FROM tmp
    INNER JOIN Genealogia g ON g.dni_mama = tmp.dni_{i}"""
    result = sqlContext.sql(query)
    count = result.count()
    if (count == 0):
        return tmp, True
    else:
        return result, False

a = getAbuelas()

i = 0
while True:
    a, fin = getHijos(a, i)
    i += 1
    if (fin):
        break


abuela = a.groupBy("dni_0","nombre").count().orderBy("count", ascending=False).first()

print("La abuela que tiene mas descendientes es " + abuela[1] + " con " + str(abuela[2]))


La abuela que tiene mas descendientes es Gavgaial con 243
